In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests

## request the page

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
request = requests.Session()
html = request.get(url)

soup = BeautifulSoup(html.content)

# the first table is the one!
table = soup.find_all('table')[0]

## check all cells and insert into a list

In [3]:
cells = []
for td in table.find_all('td'):
    cell = {}
    if td.span.text != 'Not assigned':
        cell['PostalCode'] = td.p.text[:3]
        cell['Borough'] = (td.span.text).split('(')[0]
        cell['Neighborhood'] = (((((td.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        cells.append(cell)


## Add to dataframe

In [4]:

df=pd.DataFrame(cells)
df.head()

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Queen's Park,Ontario Provincial Government


## Use geocoder to get coordinates
### (actually, not working)

In [ ]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None
lat = []
lon = []

for i, postal_code in enumerate(df['PostalCode']):
    print('Coordinates for {}'.format(postal_code, end=''))

    # loop until you get the coordinates
    while(lat_lng_coords is None):
        g = geocoder.google('{}, Toronto, Ontario'.format(postal_code))
        lat_lng_coords = g.latlng
        print('{}{}'.format(i,lat_lng_coords), end='.')

        if lat_lng_coords:
            print('')
            lat.append(lat_lng_coords[0])
            lon.append(lat_lng_coords[1])

df['latitude'] = lat
df['longitude'] = lon

### As geocoder is not working, we must import the CSV

In [5]:
import requests
url = 'https://maps.googleapis.com/maps/api/geocode/json'
params = {'sensor': 'false', 'address': 'Mountain View, CA'}
r = requests.get(url, params=params)
print(r.text)
#location = results[0]['geometry']['location']
#location['lat'], location['lng']

{
   "error_message" : "You must use an API key to authenticate each request to Google Maps Platform APIs. For additional information, please refer to http://g.co/dev/maps-no-account",
   "results" : [],
   "status" : "REQUEST_DENIED"
}



In [6]:
import pandas as pd
df2 = pd.read_csv('Geospatial_Coordinates.csv')
df2.rename(columns = {'Postal Code':'PostalCode'}, inplace = True)
df2.head()

,PostalCode,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [7]:
df3 = pd.merge(df, df2, on = "PostalCode", how = "inner")
df3.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Queen's Park,Ontario Provincial Government,43.662301,-79.389494


In [9]:
# just checking
print('df shape', df.shape)
print(df[df['PostalCode']=='M3A'])
print('-')
print('df2 shape', df2.shape)
print(df2[df2['PostalCode']=='M3A'])
print('-')
print('df3 shape', df3.shape)
print(df3[df3['PostalCode']=='M3A'])


df shape (103, 3)
  PostalCode     Borough Neighborhood
0        M3A  North York    Parkwoods
-
df2 shape (103, 3)
   PostalCode   Latitude  Longitude
25        M3A  43.753259 -79.329656
-
df3 shape (103, 5)
  PostalCode     Borough Neighborhood   Latitude  Longitude
0        M3A  North York    Parkwoods  43.753259 -79.329656


In [10]:
import json
data = json.load(open('config.json', 'r'))

CLIENT_ID = data['CLIENT_ID'] # your Foursquare ID
CLIENT_SECRET = data['CLIENT_SECRET'] # your Foursquare Secret
ACCESS_TOKEN = data['ACCESS_TOKEN'] # your FourSquare Access Token
VERSION = '20180604'
LIMIT = 30

## Now, it's time to do some research

## define class

In [106]:
class SearchFS(object):

    def __init__(self, CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN):

        self.CLIENT_ID = CLIENT_ID
        self.CLIENT_SECRET = CLIENT_SECRET
        self.ACCESS_TOKEN = ACCESS_TOKEN
        self.VERSION = '20180604'
        self.LIMIT = 30
        self.RADIUS = 500

    def venues_search(self, search_query, latitude, longitude):
        'Search for query at a given area'

        url = """https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&oauth_token={}&v={}&query={}&radius={}&limit={}""".format(
            self.CLIENT_ID,
            self.CLIENT_SECRET, 
            latitude,
            longitude,
            self.ACCESS_TOKEN,
            self.VERSION,
            search_query,
            self.RADIUS,
            self.LIMIT
            )
        return requests.get(url).json()
    
    def venue_explore(self, venue_id):
        'Explore venues'

        url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&oauth_token={}&v={}'.format(
            venue_id,
            self.CLIENT_ID,
            self.CLIENT_SECRET,
            self.ACCESS_TOKEN,
            self.VERSION)

        result = requests.get(url).json()

        if 'venue' in result['response']:
            return result['response']['venue']

    def venue_tips(self, venue_id, limit=15):
        'Venue tips'

        url = 'https://api.foursquare.com/v2/venues/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(
            venue_id,
            self.CLIENT_ID,
            self.CLIENT_SECRET,
            self.ACCESS_TOKEN,
            self.VERSION,
            limit)
        result = requests.get(url).json()
        return result['response']['tips']['items']

    def user(self, user_id):
        'User details'

        url = 'https://api.foursquare.com/v2/users/{}/?client_id={}&client_secret={}&oauth_token={}&v={}'.format(
            user_id,
            self.CLIENT_ID,
            self.CLIENT_SECRET,
            self.ACCESS_TOKEN,
            self.VERSION)
        results = requests.get(url).json()
        if 'response' in results and 'user' in results['response']:
            return results['response']['user']

    def user_tips(self, user_id, limit=15):
        'User tips'

        url = 'https://api.foursquare.com/v2/users/{}/tips?client_id={}&client_secret={}&oauth_token={}&v={}&limit={}'.format(
            user_id,
            self.CLIENT_ID,
            self.CLIENT_SECRET,
            self.ACCESS_TOKEN,
            self.VERSION,
            limit)
        results = requests.get(url).json()
        return results['response']['tips']['items']

    def location_explore(self, latitude, longitude):
        'Explore location'

        url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&ll={},{}&v={}&radius={}&limit={}'.format(
            self.CLIENT_ID,
            self.CLIENT_SECRET,
            latitude,
            longitude,
            self.VERSION,
            self.RADIUS,
            self.LIMIT)
        results = requests.get(url).json()
        return results['response']['groups'][0]['items']



searchfs = SearchFS(CLIENT_ID, CLIENT_SECRET, ACCESS_TOKEN)

In [12]:
search_postal_code='M5A'
search_query = 'restaurant'
place = df3[df3['PostalCode']==search_postal_code].squeeze()
result = searchfs.venues_search(search_query, place['Latitude'], place['Longitude'])



In [13]:
from pprint import pprint
venues = []
for i, r in enumerate(result['response']['venues']):
    venues.append(r)
    print(i, r['name'])



0 Ryan Restaurant
1 Site Of Great Canary Restaurant
2 Weheliye Restaurant
3 Morning Glory Cafe
4 White Elephant
5 Archeo


## let's do some cleaning

In [14]:
from pandas.io.json import json_normalize
venues_df = json_normalize(venues)

########
## from week2 notebook:
########

# keep only columns that include venue name, and anything that is associated with location
filtered_columns = ['name', 'categories'] + [col for col in venues_df.columns if col.startswith('location.')] + ['id']
dataframe_filtered = venues_df.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

venues_df_filt = dataframe_filtered

<ipython-input-14-d760380e5024>:2: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  venues_df = json_normalize(venues)


In [15]:
#venues_df_filt['name', 'categories']
venues_df_filt

,name,categories,address,crossStreet,lat,lng,labeledLatLngs,distance,postalCode,cc,city,state,country,formattedAddress,id
0,Ryan Restaurant,Ethiopian Restaurant,356 Queen Street E,Parliament St.,43.655724,-79.364129,"[{'label': 'display', 'lat': 43.655724, 'lng':...",325,M5A 2Z4,CA,Toronto,ON,Canada,"[356 Queen Street E (Parliament St.), Toronto ...",5b9e897ac9a5170039679335
1,Site Of Great Canary Restaurant,Breakfast Spot,NaN,NaN,43.653323,-79.357883,"[{'label': 'display', 'lat': 43.65332280645853...",245,NaN,CA,NaN,NaN,Canada,[Canada],4f872713e4b0abaa00979185
2,Weheliye Restaurant,African Restaurant,272 Parliament St,NaN,43.658536,-79.365689,"[{'label': 'display', 'lat': 43.658536, 'lng':...",626,M5A 3A4,CA,Toronto,ON,Canada,"[272 Parliament St, Toronto ON M5A 3A4, Canada]",5bf9f9f7135b39002cd82007
3,Morning Glory Cafe,Breakfast Spot,457 King St. E,Gilead Place,43.653947,-79.361149,"[{'label': 'display', 'lat': 43.65394694263529...",54,M5A 1L6,CA,Toronto,ON,Canada,"[457 King St. E (Gilead Place), Toronto ON M5A...",4ae5b91ff964a520a6a121e3
4,White Elephant,Thai Restaurant,NaN,NaN,43.655884,-79.363586,"[{'label': 'display', 'lat': 43.65588429857276...",298,NaN,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",540bea73498e234757c8b0a7
5,Archeo,Italian Restaurant,31 Trinity St.,NaN,43.650667,-79.359431,"[{'label': 'display', 'lat': 43.65066723014277...",411,M5A 3C4,CA,Toronto,ON,Canada,"[31 Trinity St., Toronto ON M5A 3C4, Canada]",4ac3e6cef964a520629d20e3


## map

In [16]:
##############
## from week2 notebook:
##############
import folium # plotting library
venues_map = folium.Map(location=[place['Latitude'], place['Longitude']], zoom_start=16) # generate map centred around the Conrad Hotel

# add a red circle marker to represent the PLACE
folium.CircleMarker(
    [place['Latitude'], place['Longitude']],
    radius=10,
    color='red',
    popup=search_postal_code,
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

# add the restaurants as blue circle markers
for lat, lng, label in zip(venues_df_filt.lat, venues_df_filt.lng, venues_df_filt.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Explore these venues

In [17]:
vexplore = []
for venue_id in venues_df_filt['id']:
    result = searchfs.venue_explore(venue_id)
    if result:
        vexplore.append(result)

print(vexplore)


D1WUD5KGBQIRT4VSPL2EITRRUX1HRLBVVUG4QT.jpg', 'width': 612, 'height': 612, 'user': {'id': '63844', 'firstName': 'Kevin', 'lastName': "O'Neill", 'gender': 'male', 'countryCode': 'US', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/63844_7Pbe5thJ_4BCEByzGUQIZ75WK_sQjaqDChnPxjbUUnFpD_0fG4asOQI5jtGftF6LIMV_thS80.jpg'}}, 'visibility': 'public'}, 'followers': {'count': 12}, 'listItems': {'count': 24, 'items': [{'id': 'v4ae5b91ff964a520a6a121e3', 'createdAt': 1335316028}]}}, {'id': '4e974226b634527c67b2a68d', 'name': 'Toronto Brunch Spots', 'description': '', 'type': 'others', 'user': {'id': '1617486', 'firstName': 'Ishy', 'lastName': 'Dee', 'gender': 'male', 'countryCode': 'US', 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/', 'suffix': '/SSWCA4B5WZLUWYEP.jpg'}}, 'editable': False, 'public': True, 'collaborative': False, 'url': '/ishdee/list/toronto-brunch-spots', 'canonicalUrl': 'https://foursquare.com/ishdee/list/toronto-brunch-spots', 'createdAt': 1318535718, 

### rating and tips

In [18]:
for v in vexplore:
    if 'rating' in v:
        rating = v['rating']
    else:
        v['rating'] = 0.0

    print('rating', v['rating'], 'tips', str(v['tips']['count']).zfill(2), v['name'], ' - ', v['categories'][0]['shortName'])



rating 0.0 tips 00 Ryan Restaurant  -  Ethiopian
rating 0.0 tips 01 Site Of Great Canary Restaurant  -  Breakfast
rating 0.0 tips 00 Weheliye Restaurant  -  African
rating 7.5 tips 16 Morning Glory Cafe  -  Breakfast
rating 0.0 tips 03 White Elephant  -  Thai
rating 7.0 tips 15 Archeo  -  Italian


### Get tips from the best place

In [23]:
max_v_rating = max(v['rating'] for v in vexplore)
best_place = [v for v in vexplore if (v['rating'] == max_v_rating)][0]
print(best_place['name'])

bp_tips = searchfs.venue_tips(best_place['id'])

'Morning Glory Cafe'

In [22]:
bp_tips

[{'id': '4ec0116c61af06192b615493',
  'createdAt': 1321210220,
  'text': 'One of the best breakfast/brunch places in the city. Affordable and good. My favourite is the Breakfast Butty. Try it...',
  'type': 'user',
  'canonicalUrl': 'https://foursquare.com/item/4ec0116c61af06192b615493',
  'likes': {'count': 0, 'groups': []},
  'like': False,
  'logView': True,
  'agreeCount': 1,
  'disagreeCount': 0,
  'todo': {'count': 0},
  'user': {'id': '4001465',
   'firstName': 'Jeremy',
   'lastName': 'Heng',
   'gender': 'male',
   'countryCode': 'CA',
   'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
    'suffix': '/BSG5PQE3XHYQWXCJ.jpg'}}}]

In [20]:

tips_df = pd.DataFrame(bp_tips)
filtered_columns = ['text', 'agreeCount', 'disagreeCount', 'id', 'user']
tips_filtered = tips_df.loc[:, filtered_columns]
tips_filtered.head()

,text,agreeCount,disagreeCount,id,user
0,One of the best breakfast/brunch places in the...,1,0,4ec0116c61af06192b615493,"{'id': '4001465', 'firstName': 'Jeremy', 'last..."


In [24]:
# returned only 1 tip (instead of 16)

from datetime import datetime
print(bp_tips[0].keys())
print('-')
cols = ['text', 'agreeCount', 'disagreeCount', 'id', 'user.firstName', 'user.lastName']
for t in bp_tips:
    print(datetime.fromtimestamp(t['createdAt']), t['text'], ' - ', t['user']['firstName'], t['user']['lastName'])
    

dict_keys(['id', 'createdAt', 'text', 'type', 'canonicalUrl', 'likes', 'like', 'logView', 'agreeCount', 'disagreeCount', 'todo', 'user'])
-
2011-11-13 15:50:20 One of the best breakfast/brunch places in the city. Affordable and good. My favourite is the Breakfast Butty. Try it...  -  Jeremy Heng


## user details

In [33]:
user_id = bp_tips[0]['user']['id']
user = searchfs.user(user_id)
user

{'id': '4001465',
 'firstName': 'Jeremy',
 'lastName': 'Heng',
 'gender': 'male',
 'countryCode': 'CA',
 'canonicalUrl': 'https://foursquare.com/user/4001465',
 'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
  'suffix': '/BSG5PQE3XHYQWXCJ.jpg'},
 'friends': {'count': 75,
  'groups': [{'type': 'friends',
    'name': 'Mutual friends',
    'count': 0,
    'items': []},
   {'type': 'others',
    'name': 'Other friends',
    'count': 75,
    'items': [{'id': '2275021',
      'firstName': 'Evan',
      'lastName': 'R.',
      'gender': 'male',
      'countryCode': 'US',
      'photo': {'prefix': 'https://fastly.4sqi.net/img/user/',
       'suffix': '/CI4GSGAW4J4JGNKO.jpg'},
      'tips': {'count': 0},
      'lists': {'groups': [{'type': 'created', 'count': 2, 'items': []}]},
      'homeCity': 'New York, NY',
      'bio': '',
      'contact': {}},
     {'id': '4782065',
      'firstName': 'Boon',
      'lastName': 'Poey Ling',
      'gender': 'female',
      'countryCode': 'MY',
   

In [43]:
pprint(user.keys())
print(user['firstName'], user['lastName'], user['gender'])

dict_keys(['id', 'firstName', 'lastName', 'gender', 'countryCode', 'canonicalUrl', 'photo', 'friends', 'tips', 'homeCity', 'bio', 'contact', 'photos', 'checkinPings', 'pings', 'type', 'mayorships', 'checkins', 'lists', 'blockedStatus', 'lenses'])
Jeremy Heng male


In [47]:
from IPython.display import Image 
#from IPython.core.display import HTML 
imgsrc = '{}540x920{}'.format(user['photo']['prefix'], user['photo']['suffix'])
Image(url=imgsrc)

In [65]:
usertips = searchfs.user_tips(user_id)
usertips

[{'id': '4ec0116c61af06192b615493',
  'createdAt': 1321210220,
  'text': 'One of the best breakfast/brunch places in the city. Affordable and good. My favourite is the Breakfast Butty. Try it...',
  'type': 'user',
  'canonicalUrl': 'https://foursquare.com/item/4ec0116c61af06192b615493',
  'likes': {'count': 0, 'groups': []},
  'like': False,
  'logView': True,
  'agreeCount': 1,
  'disagreeCount': 0,
  'todo': {'count': 0},
  'venue': {'id': '4ae5b91ff964a520a6a121e3',
   'name': 'Morning Glory Cafe',
   'location': {'address': '457 King St. E',
    'crossStreet': 'Gilead Place',
    'lat': 43.653946942635294,
    'lng': -79.36114884214422,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.653946942635294,
      'lng': -79.36114884214422}],
    'postalCode': 'M5A 1L6',
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['457 King St. E (Gilead Place)',
     'Toronto ON M5A 1L6',
     'Canada']},
   'categories': [{'id':

In [86]:
usertips_df = pd.DataFrame(usertips)
usertips_df['venuename'] = [v['name'] for v in usertips_df.venue]
usertips_df[['id', 'text', 'agreeCount', 'disagreeCount', 'venuename']]


,id,text,agreeCount,disagreeCount,venuename
0,4ec0116c61af06192b615493,One of the best breakfast/brunch places in the...,1,0,Morning Glory Cafe
1,4dacaa9c1e72c4e8dcacbbc1,Don't attempt to buy tickets here on a Monday,0,0,TO TIX


## Let's explore another area

In [152]:
location = df3.iloc[[7]]
print(location)
explore = searchfs.location_explore(float(location['Latitude']), float(location['Longitude']))


  PostalCode     Borough     Neighborhood   Latitude  Longitude
7        M3B  North York  Don Mills North  43.745906 -79.352188


In [158]:
dataframe = pd.json_normalize(explore)
# filter columns
filtered_columns = ['venue.name', 'venue.categories'] + [col for col in dataframe.columns if col.startswith('venue.location.')] + ['venue.id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# filter the category for each row
dataframe_filtered['venue.categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean columns
dataframe_filtered.columns = [col.split('.')[-1] for col in dataframe_filtered.columns]

dataframe_filtered.head(10)

,name,categories,address,lat,lng,labeledLatLngs,distance,cc,city,state,country,formattedAddress,crossStreet,postalCode,neighborhood,id
0,Island Foods,Caribbean Restaurant,1310 Don Mills Rd.,43.745866,-79.346035,"[{'label': 'display', 'lat': 43.74586607398898...",494,CA,Toronto,ON,Canada,"[1310 Don Mills Rd., Toronto ON, Canada]",NaN,NaN,NaN,4b0aed06f964a520202a23e3
1,Baretto Caffé,Café,1262 Don Mills Rd,43.744456,-79.346460,"[{'label': 'display', 'lat': 43.74445646349851...",488,CA,Toronto,ON,Canada,"[1262 Don Mills Rd, Toronto ON, Canada]",NaN,NaN,NaN,50c3ae15e4b08c3b596c6069
2,LA Fitness,Gym,1380 Don Mills Road,43.747665,-79.347077,"[{'label': 'display', 'lat': 43.74766522888878...",455,CA,Toronto,ON,Canada,"[1380 Don Mills Road (Mallard Rd), Toronto ON ...",Mallard Rd,M3B 2X2,NaN,4c18e819d4d9c9284e19f029
3,Gonoe Sushi,Japanese Restaurant,"1310 Don Mills Rd, Unit 1",43.745737,-79.345991,"[{'label': 'display', 'lat': 43.74573681959921...",498,CA,Toronto,ON,Canada,"[1310 Don Mills Rd, Unit 1 (at Bond Ave.), Tor...",at Bond Ave.,M3B 2W6,NaN,4b639286f964a5203c842ae3
4,Bond Park,Baseball Field,NaN,43.745374,-79.351245,"[{'label': 'display', 'lat': 43.74537356928136...",96,CA,Toronto,ON,Canada,"[Toronto ON, Canada]",Bond Ave. & Scarsdale Rd.,NaN,NaN,4bf97f505ec320a18a598ad3
5,Elite Basketball Camps,Athletics & Sports,B-20 Scarsdale Road,43.748790,-79.354228,"[{'label': 'display', 'lat': 43.748789728, 'ln...",360,CA,Toronto,ON,Canada,"[B-20 Scarsdale Road, Toronto ON M3B 2R2, Canada]",NaN,M3B 2R2,NaN,57e7d6bd498e0d78f21a4169
6,Wooffles & Cream,Dessert Shop,A212 - 1390 Don Mills Road,43.748159,-79.348235,"[{'label': 'display', 'lat': 43.748159, 'lng':...",404,CA,Toronto,ON,Canada,"[A212 - 1390 Don Mills Road (Leslie Street), T...",Leslie Street,M3B 3N1,Sunnybrook - York Mills,5f566cd8e9fe7c29dc6e7b13


In [159]:
latitude  = float(location['Latitude'])
longitude = float(location['Longitude'])

# from DS0701EN-2-2-1 notebook
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15) # generate map centred around Ecco

# add Ecco as a red circle mark
folium.CircleMarker(
    [latitude, longitude],
    radius=10,
    popup='Ecco',
    fill=True,
    color='red',
    fill_color='red',
    fill_opacity=0.6
    ).add_to(venues_map)


# add popular spots to the map as blue circle markers
for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        fill=True,
        color='blue',
        fill_color='blue',
        fill_opacity=0.6
        ).add_to(venues_map)

# display map
venues_map